In [14]:
#Import necessary modules
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib inline
import time

#Define arrays for step sizes and errors
h_arr = []
error_arr = []
n_arr = [10, 100, 1000, 2000, 5000, 10000]

for a in range(0,len(n_arr)):

    #Define parameters
    n = n_arr[a] #number of internal nodes in each direction (number of unknowns)
    N = n+2 #total number of nodes in each direction (includes boundaries)
    h = 1/(n+1) #step size

    #Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
    x = np.linspace(h, 1-h, n)
    y = np.linspace(h, 1-h, n)

    #Create internal mesh (excludes boundaries)
    X, Y = np.meshgrid(x, y, indexing='ij')

    #Define F 
    F = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y)  

    #Define tridiagonal matrix T
    diagonals = [[-2],[1],[1]]
    T = np.multiply(-1/h**2, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])

    for i in range(0,n):
        eigvals[i] = (T[0][0] - T[0][0]*np.cos((i+1)*np.pi/(n+1)))
        for j in range(0,n):
            eigvecs[i][j] = np.sqrt(2/(n+1))*np.sin(((i+1)*(j+1)*np.pi)/(n+1))


    #Set eigenvalues as diagonal matrix
    diag = diags(eigvals, 0, shape=(n, n)).toarray()

    #Set basis matrix M, with columns as the eigenvectors
    P = eigvecs

    #Define F_hat = P^-1 * F * P
    F_hat = np.matmul(np.matmul(np.linalg.inv(P),F),P)

    #Calculate U_hat, u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    U_hat = np.empty([n,n])
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0

    #Calculate solution U = M * U^hat * M^-1
    U = np.matmul(np.matmul(P,U_hat),np.linalg.inv(P))

    #Compute exact solution for comparison
    U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y)

    #Compute error at each node
    error_inf = 0
    for i in range(0,n):
        for j in range(0,n):
            if np.absolute(U_exact[i][j] - U[i][j]) > error_inf:
                error_inf = np.absolute(U_exact[i][j] - U[i][j])

    h_arr.append(h)
    error_arr.append(error_inf)
        





In [16]:
eoc_arr = []
for i in range(1, len(h_arr)):
    eoc = np.log(error_arr[i]/error_arr[i-1])/np.log(h_arr[i]/h_arr[i-1])
    eoc_arr.append(eoc)
    
print(eoc_arr)

[1.9927004823050405, 1.9999202057404843, 2.000013226161257, 1.9897341572441638, 2.00258342131879]


In [13]:
import numpy as np
#Kronecker product
eoc = []
err_arr = [0.0066868, 8.0611e-5, 8.2082e-7, 2.0540e-7]
h_arr = [1/11, 1/101, 1/1001, 1/2001]
for i in range(1,len(err_arr)):
    eoc.append(np.log(err_arr[i]/err_arr[i-1])/np.log(h_arr[i]/h_arr[i-1]))
print('Kronecker product: ', eoc)

#Scipy solve_sylvester
eoc = []
err_arr = [0.0066868, 8.0611e-5, 8.2083e-7, 2.0558e-7, 4.3474e-8]
h_arr = [1/11, 1/101, 1/1001, 1/2001, 1/5001]
for i in range(1,len(err_arr)):
    eoc.append(np.log(err_arr[i]/err_arr[i-1])/np.log(h_arr[i]/h_arr[i-1]))
print('\nScipy solve_syl: ', eoc)

#Similarity transformation (numpy eigs)
eoc = []
err_arr = [0.0066868, 8.0611e-5, 8.2083e-7, 2.0560e-7, 4.3041e-8, 1.5349e-9]
h_arr = [1/11, 1/101, 1/1001, 1/2001, 1/5001, 1/10001]
for i in range(1,len(err_arr)):
    eoc.append(np.log(err_arr[i]/err_arr[i-1])/np.log(h_arr[i]/h_arr[i-1]))
print('\nSimilarity trans (numpy): ', eoc)

#Similarity transformation (explicit eigs)
eoc = []
err_arr = [0.0066868, 8.0610e-5, 8.2082e-7, 2.0541e-7, 3.3196e-8, 8.2857e-9]
h_arr = [1/11, 1/101, 1/1001, 1/2001, 1/5001, 1/10001]
for i in range(1,len(err_arr)):
    eoc.append(np.log(err_arr[i]/err_arr[i-1])/np.log(h_arr[i]/h_arr[i-1]))
print('\nSimilarity trans (explicit) ', eoc)

#Bartels-stewart
eoc = []
err_arr = [0.0066868, 8.0611e-5, 8.2083e-7]
h_arr = [1/11, 1/101, 1/1001]
for i in range(1,len(err_arr)):
    eoc.append(np.log(err_arr[i]/err_arr[i-1])/np.log(h_arr[i]/h_arr[i-1]))
print('\nScipy solve_syl: ', eoc)

Kronecker product:  [1.9926958743013887, 1.9999206630573918, 2.00007136755125]

Scipy solve_syl:  [1.9926958743013887, 1.9999153514586772, 1.998824308756417, 1.6961656512877115]

Similarity trans (numpy):  [1.9926958743013887, 1.9999153514586772, 1.998683860709652, 1.7071998112990345, 4.810188885859233]

Similarity trans (explicit)  [1.9927014692818017, 1.9999152544654404, 2.000001080279963, 1.9897345276531921, 2.0026027928268824]

Scipy solve_syl:  [1.9926958743013887, 1.9999153514586772]


In [5]:
import numpy as np
np.log(1.2848e-8/5.1378e-8)/np.log((1/8001)/(1/4001))

1.9999675336259868